In [1]:
import pandas as pd
import itertools
from pulp import LpProblem, LpVariable,LpMaximize, lpSum,\
    LpStatus, LpConstraint, LpAffineExpression
import math


### Load data

In [2]:
roomsDf = pd.read_excel("input_data/2012_data/rooms-KR.xlsx")
classesDf = pd.read_excel("input_data/2012_data/class-KR-UPDATED.xlsx")

classesDf.columns =["_".join(x.lower().split(" ")) 
                    for x in classesDf.columns]
roomsDf.columns =["_".join(x.lower().split(" ")) 
                    for x in roomsDf.columns]
classesDf = classesDf.reset_index()
classesDf = classesDf.rename(index=str, columns={"index":"course_id"})
classesDf = classesDf[classesDf.days!="TBA"]

In [3]:

coursesDf = classesDf[["course_id","days","time","am/pm","max_enrollment"]]
coursesDf.loc[:,"time"] = coursesDf.time.str.split("-")
coursesDf.loc[:,"start"] = coursesDf.time.map(lambda x:x[0])
coursesDf.loc[:,"end"] = coursesDf.time.map(lambda x:x[1])
coursesDf = coursesDf[["course_id","days","start","end","am/pm","max_enrollment"]]
# coursesDf["start"] = coursesDf.start.map(lambda x:'03:00' if x=='1500' else x)
# coursesDf["end"] = coursesDf.end.map(lambda x:'04:50' if x=='1650' else x)

/Users/kshetrajna/dev/univeristy-scheduling/venv/lib/python3.6/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s
/Users/kshetrajna/dev/univeristy-scheduling/venv/lib/python3.6/site-packages/pandas/core/indexing.py:845: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = _infer_fill_value(value)


In [4]:
from datetime import datetime
# date_string = '02:00 PM'
format_str = '%I:%M %p'
# my_date = datetime.strptime(date_string, format_str)

coursesDf.loc[:,"start_time"] = coursesDf.apply(
    lambda x: datetime.strptime("{} {}".format(x["start"],x["am/pm"]),format_str),
                               axis=1)

In [5]:
def make_end_time(x):
    end_str = x["end"]
    format_str = '%I:%M %p'
    new_end = datetime.strptime("{} AM".format(x["end"]), format_str)
    if new_end > x["start_time"]:
        return new_end
    return datetime.strptime("{} PM".format(x["end"]), format_str)

coursesDf.loc[:, "end_time"] = coursesDf.apply(lambda x:make_end_time(x),axis=1)

min_time = coursesDf.start_time.min()

def get_relative_minutes(x):
    time_delta = x-min_time
    return int(time_delta.total_seconds() / 60)

coursesDf["relative_start"] = coursesDf.start_time.map(lambda x:get_relative_minutes(x))
coursesDf["relative_end"] = coursesDf.end_time.map(lambda x:get_relative_minutes(x))

In [6]:
session_length = 30
coursesDf.loc[:,"start_session"] = coursesDf["relative_start"].map(lambda x:math.floor(x/session_length))
coursesDf.loc[:,"end_session"] = coursesDf["relative_end"].map(lambda x:math.floor(x/session_length))
coursesDf.loc[:,"no_of_sessions"] = coursesDf["end_session"]-coursesDf["start_session"]+1
coursesDf["days"] = coursesDf.days.map(lambda x: [i for i in x])


In [7]:
d = {
    "M":0,
    "T":1,
    "W":2,
    "R":4,
    "F":5,
    "S":6
}
coursesDf["days"] = coursesDf.days.map(lambda x:[d[i] for i in x] )
# coursesDf[coursesDf.days.map(lambda x:"B" in x)]

In [8]:
coursesDf = coursesDf[["course_id","days","start_session","no_of_sessions","max_enrollment"]]

In [9]:
classroomsDf = roomsDf.reset_index().rename(index=str, columns={"index":"room_id"})
classroomsDf = classroomsDf[["room_id","capacity_number"]]

In [10]:
classroomsDf.head()

,room_id,capacity_number
0,0,241
1,1,115
2,2,73
3,3,49
4,4,27


In [11]:
coursesDf.head()

,course_id,days,start_session,no_of_sessions,max_enrollment
0,0,[5],12,2,76
1,1,"[1, 4]",6,3,55
2,2,"[1, 4]",9,3,55
3,3,"[1, 4]",6,3,55
4,4,"[1, 4]",12,3,50


### Create model

In [12]:
model = LpProblem("fit_scheduling_problem", LpMaximize)


### Define decsison variable

In [13]:
variable_dict = dict()
# Variable is a function of class, class_room and timeslots 
max_session = 0
for courseRow in coursesDf.values:
    course_id =  courseRow[0]
    days = courseRow[1]
    start_session = courseRow[2]
    no_of_sessions = courseRow[3]
    max_enrollment = courseRow[4]
    all_sessions = [start_session+i for i in range(no_of_sessions)]
    # variable is x_{course_id}_{room_id}_{day}_{time_slot}
    feasible_rooms = classroomsDf[classroomsDf.capacity_number>=max_enrollment].room_id.unique().tolist()
    for i in itertools.product([course_id], feasible_rooms, days, all_sessions):
        max_session = max(max_session, i[3])
        variable_dict[i] = LpVariable("x_{}_{}_{}_{}".format(i[0],i[1],i[2],i[3]),
                                      lowBound=0,
                                      upBound=1,
                                      cat='Integer'
                                     )

### variable name format is x_{course_id}_{room_id}_{day}_{time_slot}

## Add constraints

### Constraint 1
All classes have exactly one room scheduled at the starting slot for that class on each day


In [14]:
coursesDf.head()

,course_id,days,start_session,no_of_sessions,max_enrollment
0,0,[5],12,2,76
1,1,"[1, 4]",6,3,55
2,2,"[1, 4]",9,3,55
3,3,"[1, 4]",6,3,55
4,4,"[1, 4]",12,3,50


In [15]:
classrooms = classroomsDf.room_id.unique().tolist()
days = d.values()
sessions = [x for x in range(max_session+1)]

In [16]:
def create_start_class_constraint_cache(class_index, day, starting_slot, classrooms):
    variable_cache = list()
    for i in itertools.product([class_index],
                               classrooms,
                               [day],
                               [starting_slot]):
        try:
            variable_cache.append(variable_dict[i])
        except KeyError:
            pass
    return variable_cache

for class_index in coursesDf.course_id.unique():
    filteredDf = coursesDf[coursesDf.course_id==class_index]
    assert filteredDf.shape[0]==1
    days = filteredDf.days.iloc[0]
    starting_slot = filteredDf.start_session.iloc[0]
    for day in days:
        variable_cache = create_start_class_constraint_cache(class_index, day, starting_slot, classrooms)
        model+=lpSum(variable_cache)==1

### Constraint 2
If a class spans multiple days then the class should start in the same classroom every day (might want to make this an elastic constraint)

In [17]:
def create_multiple_day_repetition_constrain_cache(class_index, classrooms, days, starting_slot):
    main_cache = list()
    start_day = days[0]
    additional_days = days[1:]
    for day, classroom in itertools.product(additional_days, classrooms):
        l = list()
        try:
            l.append(variable_dict[(class_index, classroom, start_day, starting_slot)])
        except KeyError:
            pass
        if len(l)>0:
            l.append(-1*variable_dict[class_index,
                                     classroom,
                                     day,
                                     starting_slot])
            main_cache.append(l)
    return main_cache

for class_index in coursesDf.course_id.unique():
    filteredDf = coursesDf[coursesDf.course_id==class_index]
    assert filteredDf.shape[0]==1
    days = filteredDf.days.iloc[0]
    if len(days)>1:
        starting_slot = filteredDf.start_session.iloc[0]
        main_cache = create_multiple_day_repetition_constrain_cache(class_index, classrooms,
                                                                   days, starting_slot)
        assert len(main_cache)>0
        for variable_cache in main_cache:
            model+=lpSum(variable_cache)==0

### Constraint 3
If a class spans across multiple time slots then the class should be continued in the same classroom for all n time slots on the same day

In [18]:
def create_class_continuation_constraint_cache(class_index, starting_slot, no_of_sessions, day):
    additional_slots = [starting_slot+i for i in range(no_of_sessions)][1:]
    assert len(additional_slots)>0
    assert additional_slots[0]!=starting_slot, print(starting_slot, additional_slots)
    main_cache = list()
    for classroom,slot in itertools.product(classrooms, additional_slots):
        starting_variable=None
        try:
            starting_variable = variable_dict[(class_index, 
                                               classroom,
                                               day,
                                               starting_slot)]
        except KeyError:
            pass
        if starting_variable!=None:
            main_cache.append([starting_variable,
                               -1*variable_dict[
                                   class_index,
                                   classroom,
                                   day,
                                   slot
                               ]
                              ])
    return main_cache

for class_index in coursesDf.course_id.unique():
    filteredDf = coursesDf[coursesDf.course_id==class_index]
    assert filteredDf.shape[0]==1
    no_of_sessions = filteredDf.no_of_sessions.iloc[0]
    if no_of_sessions>1:
        days = filteredDf.days.iloc[0]
        starting_slot = filteredDf.start_session.iloc[0]
        for day in days:
            main_cache = create_class_continuation_constraint_cache(class_index, starting_slot, 
                                                                   no_of_sessions, day)
            for variable_cache in main_cache:
                model+=lpSum(variable_cache)==0

### Constraint 4
At any give day a class room can have only one class at at any given time slot
Making this a soft constraint with a penalty of 10 (since making this a hard constraint is making the problem space infeasible )

In [19]:
def create_non_comepete_classroom_constrain_cache(room_index, day, slot):
    variable_cache = list()
    for course_id in coursesDf.course_id.unique():
        try:
            variable_cache.append(variable_dict[course_id, room_index, day, slot])
        except KeyError:
            pass
    return variable_cache
j=0
#2753
for i in itertools.product(classrooms, days, sessions):
    if j%100==0:
        print(j)
    j+=1
    variable_cache = create_non_comepete_classroom_constrain_cache(i[0],i[1],i[2])
    c = LpAffineExpression([(variable_cache[i],1) for i in range(len(variable_cache))])
    constraint = LpConstraint(e=c,sense=-1,name='elc_{}_{}_{}'.format(i[0],i[1],i[2]),rhs=1)
    model.extend(constraint.makeElasticSubProblem(penalty=10,proportionFreeBound=0))
#     model+=lpSum(variable_cache)<=1


0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700


## Objective function
Currently just sum of all the decision vraibles subject to defined constrints above

In [20]:
model+=lpSum(variable_dict.values())

/Users/kshetrajna/dev/univeristy-scheduling/venv/lib/python3.6/site-packages/pulp/pulp.py:1353: UserWarning: Overwriting previously set objective.
  warnings.warn("Overwriting previously set objective.")


In [21]:
print(LpStatus[model.status])
model.solve()
print(LpStatus[model.status])

Not Solved
Optimal


In [33]:
nl = list()
for key in variable_dict.keys():
    nl.append([key[0],key[1],key[2],key[3], variable_dict[key].value()])

In [90]:
resDf = pd.DataFrame(nl, columns=["course_id","room_id","day","session","assigned"])
resDf.to_csv("results.csv", index=False)

In [40]:
filteredResDf = resDf[resDf.assigned==1]

In [46]:
analysisDf = filteredResDf.groupby(["day","session","room_id"]).course_id.nunique().reset_index()

In [50]:
filteredResDf[(filteredResDf.room_id==0) & (filteredResDf.session==0) & (filteredResDf.day==0)]

,course_id,room_id,day,session,assigned
24188,169,0,0,0,1.0
78715,566,0,0,0,1.0


In [52]:
coursesDf[coursesDf.course_id.isin([169, 566])]

,course_id,days,start_session,no_of_sessions,max_enrollment
169,169,"[0, 2, 5]",0,2,12
566,566,"[0, 2, 5]",0,2,20


In [57]:
analysisDf[analysisDf.course_id>1]

,day,session,room_id,course_id
0,0,0,0,2
2,0,0,3,2
11,0,1,0,2
13,0,1,3,2
22,0,2,0,4
...,...,...,...,...
2091,5,16,27,2
2093,5,16,43,2
2094,5,17,0,5
2102,5,17,27,2


In [86]:
occupancyDf = resDf.groupby(["room_id","session","day"]).assigned.max().reset_index()

,room_id,session,day,assigned
0,0,0,0,1.0
1,0,0,1,1.0
2,0,0,2,1.0
3,0,0,4,1.0
4,0,0,5,1.0
...,...,...,...,...
6732,50,24,1,0.0
6733,50,24,2,0.0
6734,50,24,4,0.0
6735,50,25,1,0.0


In [88]:
occupancyDf[(occupancyDf.assigned==0) & (occupancyDf.day==0)].sort_values(["room_id","session"])

,room_id,session,day,assigned
137,0,24,0,0.0
150,1,0,0,0.0
155,1,1,0,0.0
262,1,19,0,0.0
267,1,20,0,0.0
...,...,...,...,...
6711,50,20,0,0.0
6716,50,21,0,0.0
6721,50,22,0,0.0
6726,50,23,0,0.0


In [91]:
filteredResDf[(filteredResDf.assigned==1) & (filteredResDf.day==0) & (filteredResDf.room_id==1)].sort_values("session")

,course_id,room_id,day,session,assigned
2390,28,1,0,2,1.0
2391,28,1,0,3,1.0
19455,135,1,0,4,1.0
29165,195,1,0,4,1.0
19456,135,1,0,5,1.0
29166,195,1,0,5,1.0
36473,235,1,0,6,1.0
36474,235,1,0,7,1.0
19443,134,1,0,8,1.0
62021,418,1,0,8,1.0


In [85]:
coursesDf[coursesDf.course_id==134]

,course_id,days,start_session,no_of_sessions,max_enrollment
134,134,"[0, 2, 5]",8,2,81


In [107]:
start_sessions_only = filteredResDf.groupby(["course_id","day"]).session.min().reset_index()

In [115]:
startDf = start_sessions_only.merge(filteredResDf, on=["course_id","day","session"], how="left")
startDf = startDf.drop(["day", "assigned"], axis=1).drop_duplicates()

In [116]:
startDf

,course_id,session,room_id
0,0,12,0
1,1,6,2
3,2,9,0
5,3,6,0
7,4,12,43
...,...,...,...
1352,641,8,31
1355,642,6,33
1358,643,15,0
1360,644,18,2


In [129]:
classMetaDf = roomsDf.reset_index().rename(index=str, columns={"index":"room_id"}).drop("number",axis=1)

In [139]:
resultsDf = classesDf[["course_id","crn","title","days",
           "instructor","max_enrollment"]].merge(startDf, on="course_id", how="inner").merge(classMetaDf,
                                                                                             on="room_id",
                                                                                            how="left")
resultsDf = resultsDf.drop(["course_id","session","room_id"],axis=1)

In [141]:
resultsDf.to_csv("inital_schedule_with_conflicts.csv", index=False)